In [ ]:
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit import assemble
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService
import numpy as np
from scipy.special import kl_div

service = QiskitRuntimeService()

qc = QuantumCircuit(2, 2)

qc.h(0) 
qc.cx(0, 1) 

qc.x(0) 
qc.z(1) 
qc.x(0) 
qc.z(1) 

qc.measure([0, 1], [0, 1]) 

simulator = AerSimulator()
tqc = transpile(qc, simulator)

result = simulator.run(tqc).result()
counts = result.get_counts() 
plot_histogram(counts) 

pre_counts = {'01' : 524, '10' : 500}
post_counts = {'01' : 509, '10' : 515}

pre_probabilities = np.array([pre_counts.get(state, 0) for state in ['01' , '10']]) / sum(pre_counts.values())
post_probabilities = np.array([post_counts.get(state, 0) for state in ['01' , '10']]) / sum(post_counts.values())


relative_entropy = np.sum(kl_div(pre_probabilities, post_probabilities))

print(f"Relative Entropy (KL Divergence): {relative_entropy}")





Relative Entropy (KL Divergence): 0.00042918045994144727
